# Deep Double Descent
Autor: Piotr Branewski

## Założenia
1. Zaznajomić się z problemem deep double descent, opisanym na slajdach powyżej i cytowanym na slajdzie tytułowym artykule.
2. Wybrać dowolny zbiór danych o niewielkim wymiarze (M~10) i próbkach (wektorach) o współrzędnych (cechach) numerycznych. Liczność zbioru też bez przesady M~5000, .dane wieloklasowe. Zbiór testowy ok 20%. Proszę by zbiory się nie powtarzały by otrzymać lepszą statystykę.
3. Skupiamy się na architekturze sieci MLP (vanilla NN ze standardowymi parametrami) i zakładamy, że ilość parametrów sieci jest stała: (a) mniejsza od ilości danych; b) w przybliżeniu równa c) dużo większa.


### Przygotowanie datasetu

import libs

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

load dataset "Letter Recognition"

- Features: 16
- Samples: use a random subset of 5000 for manageable size
- Classes: 26 letter labels

In [2]:
letter = fetch_openml('letter', version=1, as_frame=False)
X_full, y_full = letter.data, letter.target
np.random.seed(42)
indices = np.random.choice(len(X_full), 5000, replace=False)
X, y = X_full[indices], y_full[indices]

#Function to get a fresh train/test split
def get_split(random_state):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=random_state
    )
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, y_train, y_test

### Zadanie 1
Dla KAŻDEJ z trzech sieci o różnej złożoności, zbadać jak zachowują się (ACC, F1) dla jej różnych realizacji czyli: a) sieć z jedną warstwą ukrytą, 2) dwoma 3) trzema itd.. Każdy wynik powtórzyć co najmniej 3 razy i wyliczyć średnią. Dla uproszczenia możemy przyjąć że ilość neuronów w warstwach ukrytych jest taka sama. Zakładamy, że CAŁKOWITA liczba neuronów jest taka sama (jak w założeniu 3)


In [3]:
# task 1
def build_mlp(n_layers, total_neurons, random_state=None):
    neurons = max(total_neurons // n_layers, 1)
    hidden_layer_sizes = tuple([neurons] * n_layers)
    model = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        activation='relu',
        solver='adam',
        learning_rate='adaptive',
        learning_rate_init=1e-3,
        max_iter=10000,
        tol=1e-4,
        random_state=random_state,
        verbose=False
    )
    return model

results = []
total_neurons_list = [10, 20, 50]
for total_neurons in total_neurons_list:
    for n_layers in [1, 2, 3]:
        for repeat in range(3):
            print(n_layers, repeat)
            X_train, X_test, y_train, y_test = get_split(random_state=repeat)
            model = build_mlp(n_layers, total_neurons, random_state=repeat)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='macro')
            results.append({
                'total_neurons': total_neurons,
                'n_layers': n_layers,
                'repeat': repeat,
                'accuracy': acc,
                'f1_macro': f1
            })

1 0
1 1
1 2
2 0
2 1
2 2
3 0
3 1
3 2
1 0
1 1
1 2
2 0
2 1
2 2
3 0
3 1
3 2
1 0
1 1
1 2
2 0
2 1
2 2
3 0
3 1
3 2


results

In [4]:
results_df = pd.DataFrame(results)
summary = results_df.groupby(['total_neurons', 'n_layers'])[['accuracy', 'f1_macro']].agg(['mean', 'std']).reset_index()
print(summary)

  total_neurons n_layers  accuracy            f1_macro          
                              mean       std      mean       std
0            10        1  0.784000  0.021000  0.783222  0.020384
1            10        2  0.631333  0.011590  0.627433  0.012280
2            10        3  0.340667  0.119182  0.305594  0.141511
3            20        1  0.840000  0.010392  0.838942  0.010538
4            20        2  0.777000  0.007211  0.776641  0.006894
5            20        3  0.686667  0.025541  0.688821  0.028471
6            50        1  0.888000  0.013892  0.887846  0.013465
7            50        2  0.867000  0.004583  0.866333  0.004117
8            50        3  0.840000  0.016643  0.839137  0.017208


### Zadanie 2
Co zaobserwujemy zwiększając liczbę warstw przy ustalonej ilości parametrów sieci? Wysnuć hipotezę dlaczego tak się może dziać i czy istnieje jakiś element architektury którego zmiana zmieniłaby tą sytuację. Rekomenduję wykorzystanie obliczenia saturacji w poszczególnych warstwach.


In [5]:
# task 2

### Zadanie 3
W kontekście deep double descent proszę zobaczyć co dzieje się gdy analizujemy wyniki z punktu (2) dla 3 sieci o różnych złożonościach z zaszumionymi etykietami.

In [6]:
# task 3